In [1]:
ref='nb9-' 

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow as tf

import random

import matplotlib

plt.style.use('seaborn-whitegrid')
matplotlib.rcParams.update({
    'font.family': 'serif',
    'axes.titlesize':16,
    'axes.labelsize':16,
    'axes.xmargin':0.1,
    'axes.ymargin':0.1,
    'legend.fontsize':16,
    'xtick.labelsize' : 16,
    'ytick.labelsize' : 16,
    'lines.markersize': 10,
    'lines.linewidth' : 3,
    'font.size': 16
})



In [2]:
f = open("../reports/{}log.txt".format(ref), "w")
f.write("Data Log File \n")
f.write("===================== \n")

23

# Importing Data

In [3]:

dataset=pd.read_csv("../data/processed/mlp_dataset_raw_xyz.csv",index_col=False )
dataset.head()

,0,1,2,3,4,5,6,7,8,9,...,293,294,295,296,297,298,299,300,301,302
0,-426.0,-413.0,-394.0,-367.0,-340.0,-313.0,-286.0,-261.0,-235.0,-220.0,...,115.0,115.0,107.0,103.0,96.0,98.0,98.0,0.133031,1.527700e+09,52-1
1,-362.0,-362.0,-362.0,-352.0,-343.0,-326.0,-324.0,-310.0,-299.0,-292.0,...,77.0,75.0,71.0,64.0,53.0,45.0,43.0,0.328511,1.527700e+09,52-1
2,-312.0,-311.0,-307.0,-313.0,-317.0,-317.0,-320.0,-321.0,-330.0,-331.0,...,25.0,22.0,14.0,13.0,28.0,67.0,100.0,0.462072,1.527700e+09,52-1
3,-408.0,-420.0,-413.0,-408.0,-410.0,-425.0,-425.0,-413.0,-374.0,-343.0,...,108.0,107.0,108.0,114.0,115.0,108.0,103.0,0.516548,1.527700e+09,52-1
4,-343.0,-331.0,-324.0,-312.0,-306.0,-301.0,-302.0,-306.0,-312.0,-312.0,...,55.0,51.0,45.0,44.0,38.0,36.0,34.0,0.591766,1.527700e+09,52-1


In [4]:
X=dataset.iloc[:,0:300].values
y=dataset.iloc[:,300].values


In [5]:
#Split training/testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)



In [6]:
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation
import mlflow.tensorflow


In [7]:
mlflow.tensorflow.autolog()

In [8]:
epochs=20
batch_size=32

In [9]:
X_train.shape[1]

300

In [10]:
model = Sequential()
model.add(Dense(128, input_shape=(X.shape[1],)))
model.add(Activation("relu"))
model.add(Dense(1))
model.add(Activation("linear"))
model.compile(loss='mean_squared_error' , optimizer="adam", metrics=["mean_squared_error"])

2022-01-04 20:42:36.426738: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:925] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-01-04 20:42:36.434859: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudnn.so.8'; dlerror: libcudnn.so.8: cannot open shared object file: No such file or directory
2022-01-04 20:42:36.434881: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1850] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2022-01-04 20:42:36.435765: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (o

In [11]:
history = model.fit(X_train,y_train, batch_size=batch_size, epochs=epochs, verbose=1, validation_split=0.1)

2022/01/04 20:42:36 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '15ed0cae86664ef5a119e268d785068c', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current tensorflow workflow


Epoch 1/20
385/385 [==============================] - 1s 3ms/step - loss: 1757.2500 - mean_squared_error: 1757.2500 - val_loss: 548.8774 - val_mean_squared_error: 548.8774
Epoch 2/20
385/385 [==============================] - 1s 2ms/step - loss: 514.8456 - mean_squared_error: 514.8456 - val_loss: 343.3074 - val_mean_squared_error: 343.3074
Epoch 3/20
385/385 [==============================] - 1s 2ms/step - loss: 349.8147 - mean_squared_error: 349.8147 - val_loss: 1018.5965 - val_mean_squared_error: 1018.5965
Epoch 4/20
385/385 [==============================] - 1s 2ms/step - loss: 468.2578 - mean_squared_error: 468.2578 - val_loss: 396.3220 - val_mean_squared_error: 396.3220
Epoch 5/20
385/385 [==============================] - 1s 3ms/step - loss: 293.0419 - mean_squared_error: 293.0419 - val_loss: 133.4444 - val_mean_squared_error: 133.4444
Epoch 6/20
385/385 [==============================] - 1s 2ms/step - loss: 98.0169 - mean_squared_error: 98.0169 - val_loss: 148.8680 - val_mean_sq

2022-01-04 20:42:52.712535: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: /tmp/tmpwq0ywwh8/model/data/model/assets


In [12]:
score = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=1)

211/211 [==============================] - 0s 1ms/step - loss: 13.5574 - mean_squared_error: 13.5574


In [13]:
f.close()